In [15]:
import sys

sys.path.append('../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport, VaryingTransport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set

In [16]:
_time_intervals = 30  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 12  # Number of execution scenarios                     (chi)

init = [1]

loc1_avail_list = init+[math.ceil(random.uniform(0,0.75)*100)/100.0 for _ in range (11)]

transport_varying_factor = pandas.DataFrame(data={('loc1', 'loc2', 'truck50'): init*3+[0]*(_exec_scenarios-3),
                                                ('loc1','loc3', 'truck30'): init*3+[0]*(_exec_scenarios-3),
                                                ('loc2','loc4', 'truck50'): init*6+[0]*(_exec_scenarios-6),
                                                ('loc3','loc4', 'truck20'): init*6+[0]*(_exec_scenarios-6),
                                                ('loc6', 'loc4', 'truck50'): init*6+[0]*(_exec_scenarios-6),
                                                ('loc4', 'loc5', 'truck100'): init*6+[0]*(_exec_scenarios-6),
                                                ('loc4', 'loc7', 'truck40'): init*6+[0]*(_exec_scenarios-6),
                                                ('loc7', 'loc4', 'truck40'): init*9+[0]*(_exec_scenarios-9),
                                                ('loc7', 'loc5','truck40'): init*9+[0]*(_exec_scenarios-9)
                                                })

M = 1e6  # Big M

# Define temporal scales
scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])

demand_factor = [1] * _exec_scenarios

In [17]:
# ======================================================================================================================
# Declare resources/commodities
# ======================================================================================================================

com1_pur = Resource(name='com1_pur', cons_max=200, block={'imp': 1, 'urg': 1}, price=0.00,
                    label='Commodity 1 consumed from outside the system',
                    varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])

com1_in = Resource(name='com1_in', label='Commodity 1 received')

com1_stored10 = Resource(name='com1_stored10', store_max=10, label='Commodity 1 in storage10')
com1_stored20 = Resource(name='com1_stored20', store_max=20, label='Commodity 1 in storage20')
com1_stored50 = Resource(name='com1_stored50', store_max=50, label='Commodity 1 in storage50')

com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')

com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')

com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True,
                     label='Commodity 1 sold to outside the system')

# com2_pur = Resource(name='com2_pur', cons_max=200, block={'imp': 1, 'urg': 1}, price=50,
#                     label='Commodity 2 consumed from outside the system', varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])
# 
# com2 = Resource(name='com2', block={'imp': 1, 'urg': 1}, label='Commodity 2')
# 
# com2_sold = Resource(name='com2_sold', revenue=80.00, demand=True, sell=True, label='Commodity 2 sold to outside the system')

In [18]:
# ======================================================================================================================
# Declare processes/storage capacities
# ======================================================================================================================
com1_procure = Process(name='procure com1', prod_max=M, conversion={com1_pur: -1, com1_in: 1}, capex=0, vopex=0,
                       fopex=0,
                       label='Procure com1')
com1_sell = Process(name='sell com1', prod_max=M, conversion={com1_out: -1, com1_sold: 1}, capex=0, vopex=0, fopex=0,
                    label='Sell com1')

com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=M, conversion={com1_loc1_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 1')
com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=M, conversion={com1_loc2_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 2')
com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=M, conversion={com1_loc3_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 3')
com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=M, conversion={com1_loc4_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 4')
com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=M, conversion={com1_loc6_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 6')
com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=M, conversion={com1_loc7_out: -1, com1_in: 1}, capex=0,
                            vopex=0, fopex=0,
                            label='Commodity 1 received from location 7')

com1_process = Process(name='com1_process', prod_max=M, conversion={com1_in: -1, com1_out: 1}, capex=750, vopex=50,
                       fopex=0,
                       label='Process the commodity through the location')

com1_store10 = Process(name='com1_store10', conversion={com1_in: -1, com1_stored10: 1}, store_max=10 / _coms,
                       prod_max=M, capex=1000, vopex=20,
                       label="Storage capacity of 10 units")
com1_discharge10 = Process(name='com1_discharge10', conversion={com1_stored10: -1, com1_out: 1},
                           label='Discharge process from storage')

com1_store20 = Process(name='com1_store20', conversion={com1_in: -1, com1_stored20: 1}, store_max=20 / _coms,
                       prod_max=M, capex=2000, vopex=15,
                       label="Storage capacity of 20 units")
com1_discharge20 = Process(name='com1_discharge20', conversion={com1_stored20: -1, com1_out: 1},
                           label='Discharge process from storage')

com1_store50 = Process(name='com1_store50', conversion={com1_in: -1, com1_stored50: 1}, store_max=50 / _coms,
                       prod_max=M, capex=5000, vopex=10,
                       label="Storage capacity of 50 units")
com1_discharge50 = Process(name='com1_discharge50', conversion={com1_stored50: -1, com1_out: 1},
                           label='Discharge process from storage')

com1_loc1_send = Process(name='com1_loc1_send', prod_max=M, conversion={com1_out: -1, com1_loc1_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')
com1_loc2_send = Process(name='com1_loc2_send', prod_max=M, conversion={com1_out: -1, com1_loc2_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')
com1_loc3_send = Process(name='com1_loc3_send', prod_max=M, conversion={com1_out: -1, com1_loc3_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')
com1_loc4_send = Process(name='com1_loc4_send', prod_max=M, conversion={com1_out: -1, com1_loc4_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')
# com1_loc5_send = Process(name='com1_loc5_send', prod_max=M, conversion={com1_out:-1, com1_loc5_out:1}, capex=0, vopex=0, fopex=0,
#                             label='Send commodity one from location 1')
com1_loc6_send = Process(name='com1_loc6_send', prod_max=M, conversion={com1_out: -1, com1_loc6_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')
com1_loc7_send = Process(name='com1_loc7_send', prod_max=M, conversion={com1_out: -1, com1_loc7_out: 1}, capex=0,
                         vopex=0, fopex=0,
                         label='Send commodity one from location 1')

In [19]:
# ======================================================================================================================
# Declare locations/warehouses
# ======================================================================================================================
loc1 = Location(name='loc1', processes={com1_procure, com1_process, com1_store20, com1_discharge20, com1_loc1_send},
                label="Location 1", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc2 = Location(name='loc2',
                processes={com1_receive_loc1, com1_process, com1_store20, com1_discharge20, com1_loc2_send},
                label="Location 2", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc3 = Location(name='loc3',
                processes={com1_receive_loc1, com1_process, com1_store20, com1_discharge20, com1_loc3_send},
                label="Location 3", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc4 = Location(name='loc4',
                processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc6, com1_process, com1_store50,
                           com1_discharge50, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc5 = Location(name='loc5',
                processes={com1_receive_loc2, com1_receive_loc4, com1_receive_loc7, com1_process, com1_store50,
                           com1_sell, com1_discharge50}, label="Location 5", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc6 = Location(name='loc6', processes={com1_procure, com1_process, com1_store10, com1_discharge10, com1_loc6_send},
                label="Location 6", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

loc7 = Location(name='loc7',
                processes={com1_receive_loc4, com1_process, com1_store20, com1_discharge20, com1_loc7_send},
                label="Location 7", scales=scales, demand_scale_level=2,
                capacity_scale_level=1, availability_scale_level=1)

In [20]:
# ======================================================================================================================
# Declare transport/trucks
# ======================================================================================================================
truck20 = Transport(name='truck20', resources=[com1_loc3_out], trans_max=20,
                    label='Truck with maximum capacity of 20 units', capex=200, vopex=15, varying=[VaryingTransport.DETERMINISTIC_CAPACITY])
truck30 = Transport(name='truck30', resources=[com1_loc1_out, com1_loc2_out], trans_max=30,
                    label='Truck with maximum capacity of 30 units', capex=300, vopex=12, varying=[VaryingTransport.DETERMINISTIC_CAPACITY])
truck40 = Transport(name='truck40', resources=[com1_loc4_out, com1_loc7_out], trans_max=40,
                    label='Truck with maximum capacity of 40 units', capex=400, vopex=10, varying=[VaryingTransport.DETERMINISTIC_CAPACITY])
truck50 = Transport(name='truck50', resources=[com1_loc1_out, com1_loc2_out, com1_loc6_out], trans_max=50,
                    label='Truck with maximum capacity of 50 units', capex=500, vopex=7, varying=[VaryingTransport.DETERMINISTIC_CAPACITY])
truck100 = Transport(name='truck100', resources=[com1_loc4_out], trans_max=100,
                     label='Truck with maximum capacity of 100 units', capex=1000, vopex=5, varying=[VaryingTransport.DETERMINISTIC_CAPACITY])

In [22]:
# ======================================================================================================================
# Declare network
# ======================================================================================================================
transport_matrix = [
    [[], [truck50], [truck30], [], [], [], []],  # sink: location 1
    [[truck50], [], [], [truck50], [truck30], [], []],  # sink: location 2
    [[truck30], [], [], [truck20], [], [], []],  # sink: location 3
    [[], [truck50], [truck20], [], [truck100], [truck50], [truck40]],  # sink: location 4
    [[], [truck30], [], [truck100], [], [], [truck40]],  # sink: location 5
    [[], [], [], [truck50], [], [], []],  # sink: location 6
    [[], [], [], [truck40], [truck40], [], []]  # sink: location 7
]

distance_matrix = [
    [0, 55, 196, M, M, M, M],
    [55, 0, M, 163, 112, M, 134],
    [196, M, 0, 63, M, M, M],
    [M, 163, 63, 0, 95, 117, 88],
    [M, 112, M, 95, 0, M, 134],
    [M, M, M, 117, M, 0, M],
    [M, 134, M, 88, 134, M, 0]
]

locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]

sources = list(locset)
sinks = list(locset)

transport_capacity_dict = {
    {(loc1, loc2): {truck50: transport_varying_factor[[('loc1', 'loc2','truck50')]]}}
}

network = Network(name='Network', scales=scales, source_locations=sources, sink_locations=sinks,
                  transport_matrix=transport_matrix, distance_matrix=distance_matrix)

TypeError: unhashable type: 'dict'